# Data Transformation

In [1]:
import os 
%pwd 

'd:\\pythonProjects\\Agentic_AI\\research'

In [2]:
os.chdir('../')

In [3]:
from pathlib import Path
from dataclasses import dataclass

@dataclass
class DataTransformationConfig:
    root_dir: Path 
    data_path: Path 
    tokenizer_name: Path 

In [8]:
from src.textSummerizer.constants import * 
from src.textSummerizer.utils.common import read_yaml, create_directory

class ConfigurationManager:
    def __init__(self,
                 config_path=CONFIG_FILE_PATH,
                 params_filepath=PARAMS_FILE_PATH):
        self.config=read_yaml(config_path)
        self.params=read_yaml(params_filepath)
        create_directory([self.config.artifacts_root])

    def get_data_tranformation_config(self)-> DataTransformationConfig:
        config=self.config.data_transformation
        create_directory([config.root_dir])

        data_transformation_config=DataTransformationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            tokenizer_name=config.tokenizer_name
        )
        return data_transformation_config    

In [9]:
import os 
from src.textSummerizer.logging import logger 
from transformers import AutoTokenizer
from datasets import load_from_disk


In [10]:
# data transformation component 
class DataTransformation:
    def __init__(self, config:DataTransformationConfig):
       self.config=config
       self.tokenizer=AutoTokenizer.from_pretrained(config.tokenizer_name)
    
    def convert_examples_to_featues(self,example_batch):
        input_encodings=self.tokenizer(example_batch['dialogue'], max_length=128, truncation=True)

        with self.tokenizer.as_target_tokenizer():
            target_encoding=self.tokenizer(example_batch['summary'],max_length=128,truncation=True)
        
        return {
            'input_ids': input_encodings['input_ids'],
            'attention_mask':input_encodings['attention_mask'],
            'labels':target_encoding['input_ids']
        }
    
    def convert(self):
        dataset_samsum=load_from_disk(self.config.data_path)
        dataset_samsum_pt=dataset_samsum.map(self.convert_examples_to_featues, batched=True)
        dataset_samsum_pt.save_to_disk(os.path.join(self.config.root_dir,'samsum_dataset'))


In [11]:
config=ConfigurationManager()
data_transformation_config=config.get_data_tranformation_config()
data_transformation=DataTransformation(config=data_transformation_config)
data_transformation.convert()

[2025-04-18 18:44:35,654: INFO: common: Yaml file config\config.yaml loaded successfully]
[2025-04-18 18:44:35,655: INFO: common: Yaml file params.yaml loaded successfully]
[2025-04-18 18:44:35,658: INFO: common: created directory at: artifacts]
[2025-04-18 18:44:35,659: INFO: common: created directory at: artifacts/data_transformation]


Map:   0%|          | 0/14732 [00:00<?, ? examples/s]

d:\pythonProjects\Agentic_AI\venv\Lib\site-packages\transformers\tokenization_utils_base.py:3980: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/819 [00:00<?, ? examples/s]

Map:   0%|          | 0/818 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/14732 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/819 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/818 [00:00<?, ? examples/s]